# Target Accrual Redemption Forward (TARF) Leveraged KO

This notebook demonstrates how to access and use the functionalities of **Target Accrual Redemption Forward (TARF)** which are part of our **QPS** module within LSEG Financial Analytics SDK.

**You will be able to:**
* Define TARF instrument with basic parameters
* Configure pricing parameters
* Evaluate TARF Analytics

TARF Leveraged KO - This is a variant of a regular TARF (possibly with an EKI on the OTM side), where there is also a KO barrier.

The barrier AFFECTS ONLY THE LEVERAGED CASH-FLOWS (i.e. the OTM side of the deal). The observation of the barrier can be continuous (American barrier) or discrete (on fixing dates). **Quite counterintuitively, it is typically an Up (Down) barrier for a Call (Put).**

If the KO barrier is triggered, future losses for the holder (OTM leveraged side) are KO, but any previous loss is maintained. **Triggering the KO barrier does not redeem the instrument**, which can still generate profits for the holder until it may reach the target.

As for any TARF, if the profit target is reached, the instrument redeems.

## Imports

Import the following necessary modules:

- `lseg_analytics.pricing.instruments.structured_products` - for Structured Products instruments definitions and analytics

This notebook uses external libraries **pandas, IPython**; please ensure they are installed in your Python environment (e.g. 'pip install pandas') before running the code.

In [1]:
from lseg_analytics.pricing.instruments import structured_products as sp 

import datetime as dt
import pandas as pd
import json
from IPython.display import display

## Data Preparation

To define a Structured Product instrument, in this example leveraged KO TARF, you need to follow a structured 3-step process:
* **Structured Product Definition** - Specify basic Structured Product parameters (underlying rate, payoff description, dates, notional)
* **Structured Product Instrument Definition** - Create the instrument object
* **Pricing Preferences** - Configure pricing models and parameters, optional

In [2]:
# 1. Create SP definition object

print("Step 1: Configuring instrument definition...")

TARF_definition = sp.StructuredProductsDefinition(
    deal_ccy = "EUR",
    instrument_tag = "TARF_Leveraged_KO",
    inputs = [
        sp.NameTypeValue(name="Underlying", type = "string", value="USDEUR"),
        sp.NameTypeValue(name="StartDate", type = "date", value= dt.date(2022, 1, 25)),
        sp.NameTypeValue(name="EndDate", type = "string", value="StartDate + 1Y"),
        sp.NameTypeValue(name="Frequency", type = "string", value="1M"),
        sp.NameTypeValue(name="Notional", type = "string", value="1000000"),
        sp.NameTypeValue(name="Strike", type = "string", value="0.85"),
        sp.NameTypeValue(name="Barrier", type = "string", value="0.9"),
        sp.NameTypeValue(name="Leverage", type = "string", value="2"),
        sp.NameTypeValue(name="ProfitTarget", type = "string", value="0.05"),
        sp.NameTypeValue(name="KO_Payment", type = "string", value="Settlement[t]"),
    ],
    payoff_description = [
					[
                        "ScheduleType",
                        "Schedule description",
                        "FX",
                        "Coupon",
                        "OTMAlive",
                        "Settlement",
                        "Sum",
                        "Alive",
                        "KO_Amount",
                        "Price"
                    ],
                    [
                        "AtDate",
                        "StartDate",
                        "",
                        "",
                        "1",
                        "",
                        "0",
                        "1",
                        "",
                        ""
                    ],
                    [
                        "OnSchedule",
                        "DateTable(StartDate + Frequency, EndDate, Frequency, ResetGap := 0b)",
                        "FxSpot(Underlying)",
                        "If(FX[t] < Strike, Leverage  * Notional * (FX[t] - Strike), Notional * (FX[t] - Strike))",
                        "If(FX[t] >= Barrier, 0, OTMAlive[LastDate - 1])",
                        "If(Coupon[t] >= 0, Coupon[t], Coupon[t] * OTMAlive[t])",
                        "Sum[LastDate-1] + Max(FX[t] - Strike, 0)",
                        "If(Sum[t] >= ProfitTarget, 0, Alive[LastDate - 1])",
                        "Alive[LastDate - 1] * (1 - Alive[LastDate]) * KO_Payment",
                        "Alive[t] * Settlement[t] + KO_Amount[t]"
                    ],
                    [
                        "AtDate",
                        "EndDate",
                        "FxSpot(Underlying)",
                        "If(FX[t] < Strike, Leverage  * Notional * (FX[t] - Strike), Notional * (FX[t] - Strike))",
                        "If(FX[t] >= Barrier, 0, OTMAlive[LastDate - 1])",
                        "If(Coupon[t] >= 0, Coupon[t], Coupon[t] * OTMAlive[t])",
                        "Sum[LastDate-1] + Max(FX[t] - Strike, 0)",
                        "If(Sum[t] >= ProfitTarget, 0, Alive[LastDate - 1])",
                        "Alive[LastDate - 1] * (1 - Alive[LastDate]) * KO_Payment",
                        "Alive[t] * Settlement[t] + KO_Amount[t]"
                    ]
				]
)


print("	Instrument definition configured")

# 2. Create SP instrument definition object
print("Step 2: Creating instrument definition object...")

tarf_leveraged_ko = sp.StructuredProductsDefinitionInstrument(definition = TARF_definition)
print("	Instrument definition created")


# 3. Create SP parameters object - optional
print("Step 3: Configuring pricing parameters...")

TARF_pricing_params = sp.StructuredProductsPricingParameters(
    valuation_date= dt.date(2022, 3, 16),  # Set your desired valuation date
    numerical_method = sp.GenericNumericalMethod(method="MonteCarlo"),
    models=[sp.ModelDefinition(
            underlying_code = "USDEUR",
            underlying_tag = "USDEUR",
            underlying_currency = "EUR",
            asset_class = "ForeignExchange",
            model_name= "Heston")]
)
print("	Pricing parameters configured")

Step 1: Configuring instrument definition...
	Instrument definition configured
Step 2: Creating instrument definition object...
	Instrument definition created
Step 3: Configuring pricing parameters...
	Pricing parameters configured


## Request Execution

In [3]:
# Execute the calculation using the price function
try:
    # The 'definitions' parameter accepts a list of request items for batch processing
    response = sp.price(
        definitions=[tarf_leveraged_ko],
        pricing_preferences=TARF_pricing_params,
        market_data=None,
        return_market_data=True,  # or False
        fields=None  # or specify fields as a string
    )
    errors = [a.error for a in response.data.analytics if a.error]
    if errors:
        raise Exception(errors[0].message)
    print("Pricing Execution Successful!")
except Exception as e:
    print(f"Price Calculation failed: {str(e)}")
    raise

Pricing Execution Successful!


## Results Display

### Key Sections in the `response` JSON

 - **definitions**: Instrument setup (e.g., underlying, payoff description, dates, notional), it's StructuredProductDefinition that we used. 

 - **pricingPreferences**: Valuation date, financial model, numerical method, it is StructuredProductsPricingParameters we used

 - **analytics**:
   - **tabularData**: `data`, `headers`, `statuses`
   - **cashflows**: Includes arrays and detailed `cashFlows` with discount factors
   - **description**: Instrument summary StructuredProductDefinition and also default fields not specified in the StructuredProductDefinition, but used by default in the calculation (e.g. price)
   - **greeks**: Sensitivities like `deltaAmountInDealCcy`, `gammaAmountInDealCcy`, `thetaAmountInDealCcy`, `vegaAmountInDealCcy`
   - **pricingAnalysis**: `valuationDate`, `marketDataDate`
   - **valuation**: `marketValueInDealCcy`
   - **error**: Empty if no issues

The analytics fields are useful for understanding which fields are included by default in the price function, even if they are not explicitly specified in `price` function. We detail below the description, valuation, cash-flows and greeks.

### Description

In [4]:
# Extract description from response
description = response.data.analytics[0].description

# Convert to dictionary for display
print(json.dumps(description.as_dict(), indent=4))

{
    "instrumentTag": "TARF_Leveraged_KO",
    "dealCcy": "EUR",
    "discountCurveId": "IRCurve_EUREURIBORSwapZCCurve_0001-01-01T00:00:00",
    "discountCurveName": "EUR EURIBOR Swap ZC Curve",
    "outputList": {
        "Alive": 0.741341546,
        "Coupon": 492536.773460774,
        "FX": 9.916931107,
        "KO_Amount": 54455.838925632,
        "OTMAlive": 3.194004321,
        "Price": 39153.798751709,
        "Settlement": 551016.836900749,
        "Sum": 2.177579451
    },
    "processingInformation": "'PricePercent' column was not retrieved in payoff description: related fields will not be retrieved. The following greeks could not be retrieved: Theta, Vega, Gamma, Delta. Make sure a 'PricePercent' column is specified in payoff description to compute greeks. "
}


### Valuation

In [5]:
# Extract vauation from the response
valuation = response.data.analytics[0].valuation

# Convert the dictionary to a DataFrame
df_tarf_valuation = pd.DataFrame(list(valuation.items()), columns=["Field", "Value"])

display(df_tarf_valuation)

,Field,Value
0,marketValueInDealCcy,39153.798752


### Cash Flows

In [6]:
# Extract cashflows from response
cashflows = response.data.analytics[0].cashflows["cashFlows"]

# Build dataframes for all cash flow types
df_fx = pd.DataFrame(cashflows[0]['payments']).rename(columns={'amount': cashflows[0]['legTag']})
df_coupon = pd.DataFrame(cashflows[1]['payments']).rename(columns={'amount': cashflows[1]['legTag']})
df_OTMAlive = pd.DataFrame(cashflows[2]['payments']).rename(columns={'amount': cashflows[2]['legTag']})
df_settlement = pd.DataFrame(cashflows[3]['payments']).rename(columns={'amount': cashflows[3]['legTag']})
df_sum  = pd.DataFrame(cashflows[4]['payments']).rename(columns={'amount': cashflows[4]['legTag']})
df_alive = pd.DataFrame(cashflows[5]['payments']).rename(columns={'amount': cashflows[5]['legTag']})
df_ko_amount = pd.DataFrame(cashflows[6]['payments']).rename(columns={'amount': cashflows[6]['legTag']})
df_price = pd.DataFrame(cashflows[7]['payments']).rename(columns={'amount': cashflows[7]['legTag']})

# Merge all dataframes on the 'date' column
# Add suffix to distinguish between different data sources
combined_df = pd.concat(
    [df_fx.add_suffix('_fx'), 
     df_coupon.add_suffix('_coupon'),
     df_OTMAlive.add_suffix('_OTMAlive'), 
     df_settlement.add_suffix('_settlement'), 
     df_sum.add_suffix('_sum'), 
     df_alive.add_suffix('_alive'), 
     df_ko_amount.add_suffix('_ko_amount'), 
     df_price.add_suffix('_price')], 
    axis=1
)

# Remove duplicate columns with the same values
combined_df = combined_df.loc[:, ~combined_df.T.duplicated()]

# Keep only one discountFactor column and sort the columns
combined_df = combined_df.loc[:,['date_fx', 'discountFactor_fx', 'FX_fx', 'Coupon_coupon', 'OTMAlive_OTMAlive', 'Settlement_settlement','Sum_sum',  'Alive_alive',
       'KO_Amount_ko_amount','Price_price', 'currency_fx', 'event_fx','occurence_fx']]

# Rename columns to match the desired layout
combined_df.columns = ['date', 'discountFactor', 'FX', 'Coupon', 'OTMAlive', 'Settlement', 'Sum', 'Alive', 'KO_Amount', 'Price', 'currency', 'event', 'occurrence']

# Display the combined dataframe
display(combined_df)

,date,discountFactor,FX,Coupon,OTMAlive,Settlement,Sum,Alive,KO_Amount,Price,currency,event,occurrence
0,2022-01-25,0.000000,0.000000,0.000000,1.0000,0.000000,0.000000,1.0000,0.000000,0.000000,EUR,Payoff,Historical
1,2022-02-25,1.000000,0.887390,37390.185465,1.0000,37390.185465,0.037390,1.0000,0.000000,37390.185465,EUR,Payoff,Historical
2,2022-03-25,1.000123,0.906117,56117.456667,0.3318,56117.456667,0.056117,0.3318,42552.914876,56117.456667,EUR,Payoff,Future
3,2022-04-25,1.000636,0.905213,54586.170661,0.4220,54586.170661,0.093231,0.0642,13463.980821,12455.499802,EUR,Payoff,Future
4,2022-05-25,1.001116,0.904112,52457.216959,0.2426,54128.503619,0.111883,0.0562,578.840791,-772.286943,EUR,Payoff,Future
5,2022-06-27,1.001633,0.902777,49833.053086,0.3674,51128.658066,0.148951,0.0402,79.006412,-2844.726732,EUR,Payoff,Future
6,2022-07-25,1.002044,0.901621,47692.847815,0.2230,52410.072692,0.167433,0.0398,-77.507255,-2588.917262,EUR,Payoff,Future
7,2022-08-25,1.002452,0.900025,45136.216143,0.3446,48622.138566,0.203865,0.0376,-510.360248,-4140.617062,EUR,Payoff,Future
8,2022-09-26,1.002831,0.898307,42294.540860,0.2116,50369.916646,0.221753,0.0364,-163.885625,-2912.059647,EUR,Payoff,Future
9,2022-10-25,1.003093,0.896887,39909.259058,0.3284,45961.640842,0.257729,0.0350,-499.427241,-4527.661917,EUR,Payoff,Future


### Greeks

In [7]:
# Extract Greeks from the response
greeks = response.data.analytics[0].greeks

# Convert the dictionary to a DataFrame
df_tarf_greeks = pd.DataFrame(list(greeks.items()), columns=["Greeks", "Value"])

display(df_tarf_greeks)

,Greeks,Value
0,deltaAmountInDealCcy,15019.10
1,gammaAmountInDealCcy,-9054.55
2,vegaAmountInDealCcy,-17555.10
3,thetaAmountInDealCcy,1305.38
